In [8]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'C:\\Users\\malik\\Desktop\\Disertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [100, 101, 102]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def add_noise_to_segments(ecg_segments, noise_signal, snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    for ecg_segment in ecg_segments:
        snr = 10 ** (snr_db / 10)
        noise_power = np.sum(ecg_segment ** 2) / (snr * len(ecg_segment))
        noise = np.sqrt(noise_power) * noise_signal
        noisy_segment = ecg_segment + noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear')
    clf.fit(features, labels)
    return clf

# Noise types
noises = {'EM': em_noise, 'BW': bw_noise, 'MA': ma_noise}
snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    accuracy_noisy = accuracy_score(train_labels, model_noisy.predict(features_noisy))
    accuracy_denoised = accuracy_score(train_labels, model_denoised.predict(features_denoised))
    
    results[noise_name] = {'noisy': accuracy_noisy, 'denoised': accuracy_denoised}

print(results)


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


133/133 ━━━━━━━━━━━━━━━━━━━━ 23s 169ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 26s 198ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 27s 204ms/step
{'EM': {'noisy': 0.9995279679018173, 'denoised': 0.9903233419872551}, 'BW': {'noisy': 0.9995279679018173, 'denoised': 0.9903233419872551}, 'MA': {'noisy': 0.9995279679018173, 'denoised': 0.9903233419872551}}


In [10]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [100, 101, 102]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def add_noise_to_segments(ecg_segments, noise_signal, snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    for ecg_segment in ecg_segments:
        snr = 10 ** (snr_db / 10)
        noise_power = np.sum(ecg_segment ** 2) / (snr * len(ecg_segment))
        noise = np.sqrt(noise_power) * noise_signal
        noisy_segment = ecg_segment + noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear')
    clf.fit(features, labels)
    return clf

# Noise types
noises = {'EM': em_noise, 'BW': bw_noise, 'MA': ma_noise}
snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True)
    
    # Store accuracy for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names}
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")

C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


133/133 ━━━━━━━━━━━━━━━━━━━━ 23s 169ms/step


C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


133/133 ━━━━━━━━━━━━━━━━━━━━ 27s 200ms/step


C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


133/133 ━━━━━━━━━━━━━━━━━━━━ 27s 204ms/step
Noise type: EM
Noisy data accuracies:
  N: 0.9995
  V: 1.0000
  A: 1.0000
Denoised data accuracies:
  N: 0.9903
  V: 0.0000
  A: 0.0000


Noise type: BW
Noisy data accuracies:
  N: 0.9995
  V: 1.0000
  A: 1.0000
Denoised data accuracies:
  N: 0.9903
  V: 0.0000
  A: 0.0000


Noise type: MA
Noisy data accuracies:
  N: 0.9995
  V: 1.0000
  A: 1.0000
Denoised data accuracies:
  N: 0.9903
  V: 0.0000
  A: 0.0000




C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(mapped_labels), y=mapped_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def add_noise_to_segments(ecg_segments, noise_signal, snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    for ecg_segment in ecg_segments:
        snr = 10 ** (snr_db / 10)
        noise_power = np.sum(ecg_segment ** 2) / (snr * len(ecg_segment))
        noise = np.sqrt(noise_power) * noise_signal
        noisy_segment = ecg_segment + noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Noise types
noises = {'EM': em_noise, 'BW': bw_noise, 'MA': ma_noise}
snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names}
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


756/756 ━━━━━━━━━━━━━━━━━━━━ 160s 212ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 176s 233ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 180s 238ms/step
Noise type: EM
Noisy data accuracies:
  N: 0.9967
  V: 0.7387
  A: 0.0693
  L: 0.9083
Denoised data accuracies:
  N: 0.9942
  V: 0.7437
  A: 0.0500
  L: 0.9081


Noise type: BW
Noisy data accuracies:
  N: 0.9974
  V: 0.7555
  A: 0.0691
  L: 0.9243
Denoised data accuracies:
  N: 0.9947
  V: 0.6510
  A: 0.0557
  L: 0.9286


Noise type: MA
Noisy data accuracies:
  N: 0.9974
  V: 0.7559
  A: 0.0679
  L: 0.9119
Denoised data accuracies:
  N: 0.9937
  V: 0.5963
  A: 0.0576
  L: 0.8997




In [13]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(mapped_labels), y=mapped_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Noise types
noises = {'EM': em_noise, 'BW': bw_noise, 'MA': ma_noise}
snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names}
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


756/756 ━━━━━━━━━━━━━━━━━━━━ 174s 230ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 177s 235ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 179s 237ms/step
Noise type: EM
Noisy data accuracies:
  N: 0.9966
  V: 0.7190
  A: 0.0709
  L: 0.9242
Denoised data accuracies:
  N: 0.9895
  V: 0.6810
  A: 0.0781
  L: 0.8215


Noise type: BW
Noisy data accuracies:
  N: 0.9973
  V: 0.7502
  A: 0.0668
  L: 0.9297
Denoised data accuracies:
  N: 0.9852
  V: 0.4456
  A: 0.0822
  L: 0.7231


Noise type: MA
Noisy data accuracies:
  N: 0.9968
  V: 0.7400
  A: 0.0688
  L: 0.9451
Denoised data accuracies:
  N: 0.9858
  V: 0.7538
  A: 0.0725
  L: 0.3842




In [14]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(mapped_labels), y=mapped_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Create combined noise types
combined_noises = {
    'EM+MA': em_noise + ma_noise,
    'EM+BW': em_noise + bw_noise,
    'MA+BW': ma_noise + bw_noise,
    'EM+BW+MA': em_noise + bw_noise + ma_noise
}

# Add single noises to combined noises
noises = {
    'EM': em_noise,
    'BW': bw_noise,
    'MA': ma_noise
}
noises.update(combined_noises)

snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names}
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


756/756 ━━━━━━━━━━━━━━━━━━━━ 161s 213ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 173s 229ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 174s 230ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 174s 230ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 173s 229ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 173s 229ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 173s 229ms/step
Noise type: EM
Noisy data accuracies:
  N: 0.9966
  V: 0.7190
  A: 0.0709
  L: 0.9242
Denoised data accuracies:
  N: 0.9876
  V: 0.6036
  A: 0.0226
  L: 0.3090


Noise type: BW
Noisy data accuracies:
  N: 0.9973
  V: 0.7502
  A: 0.0668
  L: 0.9297
Denoised data accuracies:
  N: 0.9873
  V: 0.6690
  A: 0.0301
  L: 0.3627


Noise type: MA
Noisy data accuracies:
  N: 0.9968
  V: 0.7400
  A: 0.0688
  L: 0.9451
Denoised data accuracies:
  N: 0.9829
  V: 0.4419
  A: 0.0123
  L: 0.2920


Noise type: EM+MA
Noisy data accuracies:
  N: 0.9966
  V: 0.7254
  A: 0.0697
  L: 0.9238
Denoised data accuracies:
  N: 0.9869
  V: 0.5878
  A: 0.0226
  L: 0.3044


Noise type: EM+BW
Noisy data accur

In [1]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input, Add
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model with skip connections
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    # Encoder
    x1 = Conv1D(64, 15, padding='same')(inp)
    x1 = LeakyReLU(alpha=0.2)(x1)

    x2 = Conv1D(128, 15, padding='same')(x1)
    x2 = LeakyReLU(alpha=0.2)(x2)

    x3 = Conv1D(256, 15, padding='same')(x2)
    x3 = LeakyReLU(alpha=0.2)(x3)

    x4 = Conv1D(512, 15, padding='same')(x3)
    x4 = LeakyReLU(alpha=0.2)(x4)

    # Decoder with skip connections
    x5 = Conv1DTranspose(256, 15, padding='same')(x4)
    x5 = LeakyReLU(alpha=0.2)(x5)
    x5 = Add()([x5, x3])

    x6 = Conv1DTranspose(128, 15, padding='same')(x5)
    x6 = LeakyReLU(alpha=0.2)(x6)
    x6 = Add()([x6, x2])

    x7 = Conv1DTranspose(64, 15, padding='same')(x6)
    x7 = LeakyReLU(alpha=0.2)(x7)
    x7 = Add()([x7, x1])

    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x7)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(mapped_labels), y=mapped_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def calculate_rmse(signal, denoised_signal):
    return np.sqrt(np.mean((signal - denoised_signal) ** 2))

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Create combined noise types
combined_noises = {
    'EM+MA': em_noise + ma_noise,
    'EM+BW': em_noise + bw_noise,
    'MA+BW': ma_noise + bw_noise,
    'EM+BW+MA': em_noise + bw_noise + ma_noise
}

# Add single noises to combined noises
noises = {
    'EM': em_noise,
    'BW': bw_noise,
    'MA': ma_noise
}
noises.update(combined_noises)

snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    # Calculate SNR and RMSE
    snr_values = [calculate_snr(ecg, denoised) for ecg, denoised in zip(ecg_segments, denoised_ecg_slices)]
    rmse_values = [calculate_rmse(ecg, denoised) for ecg, denoised in zip(ecg_segments, denoised_ecg_slices)]
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy and SNR, RMSE for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names},
        'snr': np.mean(snr_values),
        'rmse': np.mean(rmse_values)
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print(f"Average SNR after denoising: {metrics['snr']:.4f} dB")
    print(f"Average RMSE after denoising: {metrics['rmse']:.4f}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


756/756 ━━━━━━━━━━━━━━━━━━━━ 161s 212ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 177s 234ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 178s 235ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 317s 420ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 363s 481ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 363s 481ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 364s 482ms/step
Noise type: EM
Average SNR after denoising: 8.7213 dB
Average RMSE after denoising: 0.7568
Noisy data accuracies:
  N: 0.9966
  V: 0.7190
  A: 0.0709
  L: 0.9242
Denoised data accuracies:
  N: 0.9916
  V: 0.7162
  A: 0.0558
  L: 0.8044


Noise type: BW
Average SNR after denoising: 7.3092 dB
Average RMSE after denoising: 0.8149
Noisy data accuracies:
  N: 0.9973
  V: 0.7502
  A: 0.0668
  L: 0.9297
Denoised data accuracies:
  N: 0.9911
  V: 0.4650
  A: 0.0679
  L: 0.7522


Noise type: MA
Average SNR after denoising: 11.0426 dB
Average RMSE after denoising: 0.7273
Noisy data accuracies:
  N: 0.9968
  V: 0.7400
  A: 0.0688
  L: 0.9451
Denoised data accuracies:
  N: 0.9930
  V: 0.5

In [ ]:
import wfdb
import numpy as np
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input, Add
from tensorflow.keras.models import Model

# Load ECG data with correct labels from the MIT-BIH Arrhythmia Database
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model with skip connections
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    # Encoder
    x1 = Conv1D(64, 15, padding='same')(inp)
    x1 = LeakyReLU(alpha=0.2)(x1)

    x2 = Conv1D(128, 15, padding='same')(x1)
    x2 = LeakyReLU(alpha=0.2)(x2)

    x3 = Conv1D(256, 15, padding='same')(x2)
    x3 = LeakyReLU(alpha=0.2)(x3)

    x4 = Conv1D(512, 15, padding='same')(x3)
    x4 = LeakyReLU(alpha=0.2)(x4)
    
    x5 = Conv1D(1024, 15, padding='same')(x4)
    x5 = LeakyReLU(alpha=0.2)(x5)
    
    x6 = Conv1D(2048, 15, padding='same')(x5)
    x6 = LeakyReLU(alpha=0.2)(x6)
    
    x7 = Conv1D(4096, 15, padding='same')(x6)
    x7 = LeakyReLU(alpha=0.2)(x7)

    # Decoder with skip connections
    x8 = Conv1DTranspose(2048, 15, padding='same')(x7)
    x8 = LeakyReLU(alpha=0.2)(x8)
    x8 = Add()([x8, x6])

    x9 = Conv1DTranspose(1024, 15, padding='same')(x8)
    x9 = LeakyReLU(alpha=0.2)(x9)
    x9 = Add()([x9, x5])

    x10 = Conv1DTranspose(512, 15, padding='same')(x9)
    x10 = LeakyReLU(alpha=0.2)(x10)
    x10 = Add()([x10, x4])

    x11 = Conv1DTranspose(256, 15, padding='same')(x10)
    x11 = LeakyReLU(alpha=0.2)(x11)
    x11 = Add()([x11, x3])

    x12 = Conv1DTranspose(128, 15, padding='same')(x11)
    x12 = LeakyReLU(alpha=0.2)(x12)
    x12 = Add()([x12, x2])

    x13 = Conv1DTranspose(64, 15, padding='same')(x12)
    x13 = LeakyReLU(alpha=0.2)(x13)
    x13 = Add()([x13, x1])

    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x13)
    
    return Model(inp, out)

generator = build_generator()

# Load ECG and noise data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Add more records as needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)
em_noise, bw_noise, ma_noise = load_noise_data()

# Filter and map labels to integer categories
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}
mapped_labels = np.array([label_mapping.get(label, -1) for label in labels])
valid_indices = mapped_labels != -1

ecg_segments = ecg_segments[valid_indices]
mapped_labels = mapped_labels[valid_indices]

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(mapped_labels), y=mapped_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def calculate_rmse(signal, denoised_signal):
    return np.sqrt(np.mean((signal - denoised_signal) ** 2))

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Create combined noise types
combined_noises = {
    'EM+MA': em_noise + ma_noise,
    'EM+BW': em_noise + bw_noise,
    'MA+BW': ma_noise + bw_noise,
    'EM+BW+MA': em_noise + bw_noise + ma_noise
}

# Add single noises to combined noises
noises = {
    'EM': em_noise,
    'BW': bw_noise,
    'MA': ma_noise
}
noises.update(combined_noises)

snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    # Calculate SNR and RMSE
    snr_values = [calculate_snr(ecg, denoised) for ecg, denoised in zip(ecg_segments, denoised_ecg_slices)]
    rmse_values = [calculate_rmse(ecg, denoised) for ecg, denoised in zip(ecg_segments, denoised_ecg_slices)]
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    train_labels = mapped_labels[:len(features_noisy)]
    
    model_noisy = classify_heartbeats(features_noisy, train_labels)
    model_denoised = classify_heartbeats(features_denoised, train_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in train_labels
    unique_classes = np.unique(train_labels)
    class_names = [name for i, name in enumerate(['N', 'V', 'A', 'L']) if i in unique_classes]
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(train_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(train_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy and SNR, RMSE for each class
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] for class_name in class_names},
        'snr': np.mean(snr_values),
        'rmse': np.mean(rmse_values)
    }

# Output the results for each class and noise condition
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print(f"Average SNR after denoising: {metrics['snr']:.4f} dB")
    print(f"Average RMSE after denoising: {metrics['rmse']:.4f}")
    print("Noisy data accuracies:")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("Denoised data accuracies:")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.4f}")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


361/756 ━━━━━━━━━━━━━━━━━━━━ 5:43:04 52s/step

In [ ]:
import matplotlib.pyplot as plt

# Plotting function for the noisy and denoised ECG signals
def plot_ecg_signals(noise_name, ecg_segments, noisy_ecg_slices, denoised_ecg_slices, num_samples=3):
    plt.figure(figsize=(10, 10))

    for i in range(num_samples):
        # Plot original ECG
        plt.subplot(num_samples, 3, i * 3 + 1)
        plt.plot(ecg_segments[i], linewidth=0.8)
        if i == 0:
            plt.title('Original ECG')
        plt.ylim([-2, 2])
        plt.xlim([0, len(ecg_segments[i])])

        # Plot noisy ECG
        plt.subplot(num_samples, 3, i * 3 + 2)
        plt.plot(noisy_ecg_slices[i], linewidth=0.8)
        if i == 0:
            plt.title(f'Noisy ECG ({noise_name})')
        plt.ylim([-2, 2])
        plt.xlim([0, len(noisy_ecg_slices[i])])

        # Plot denoised ECG
        plt.subplot(num_samples, 3, i * 3 + 3)
        plt.plot(denoised_ecg_slices[i], linewidth=0.8)
        if i == 0:
            plt.title('Denoised ECG')
        plt.ylim([-2, 2])
        plt.xlim([0, len(denoised_ecg_slices[i])])

    plt.tight_layout()
    plt.show()

# Plotting for all noise types
for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    # Generate plots
    plot_ecg_signals(noise_name, ecg_segments, noisy_ecg_slices, denoised_ecg_slices)
